In [ ]:
import requests
import time
import json
import html
import time
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from transformers import pipeline
import sentencepiece
import pandas as pd
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import os
import re

In [ ]:
# Initialize the question generation pipeline
qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

In [ ]:
# print(content[0:50000])

In [ ]:
def load_and_regex_paragraphs(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    
    text = re.sub(r"\bFigure\s+\d+-\d+\b", "", text, flags=re.IGNORECASE)
    # Remove any number surrounded by spaces (e.g., " 42 ")
    text = re.sub(r"\d+", " ", text)
    # Optional: strip redundant whitespace
    cleaned_text = re.sub(r"\s{2,}", " ", text).strip()

    return text

In [ ]:
def load_and_split_paragraphs(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Split on one or more blank lines (handles multiple \n between paragraphs)
    import re
    pattern = r"(*Figure -*\s*\s*-\s*(\d+)\s*"
    pat = r"[ ]*(\d+[ ]*)|\(*Figure -*\s*\d+\)?"
    # paragraphs = [para.strip() for para in re.split(r'\n\s*\n', re.sub(pattern, " ", text)) if para.strip()]
    paragraphs = [para.strip().replace("- . ","").replace(" . ","") for para in re.split(r'\n\s*\n', re.sub(r"\s*\d+\s*", "", re.sub(pat, " ", text))) if para.strip()]
    
    return paragraphs


In [ ]:
# Example usage
paragraphs = load_and_split_paragraphs("/Users/rckyi/Documents/Data/Nasa-Lessons-learned-in-engineering.txt")
for i, para in enumerate(paragraphs[:500]):  # Print first 5 paragraph
    input_text = f"generate question: {para}"
    generated_question = qg_pipeline(input_text, max_length=64, do_sample=False)[0]['generated_text']
    print(f"\n--- Paragraph {i + 1} ---\n{generated_question}\n{para}")
        

In [ ]:
import json

def func(paragraph):
    # Example placeholder function — replace this with your desired logic
    return len(paragraph.split())  # returns word count

def process_paragraphs(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        paragraphs = json.load(f)
    
    for para_id, para_text in paragraphs.items():
        input_text = f"generate question: {para_text}"
        output = qg_pipeline(input_text, max_length=64, do_sample=False)[0]['generated_text']
        yield [qg_pipeline(input_text, max_length=512, do_sample=False)[0]['generated_text'], para_text]

# Example usage:
json_file_path = "/Users/rckyi/Documents/Data/paragraphs_with_ids.json"  # or your full path
results = list(process_paragraphs(json_file_path))


In [ ]:
from happytransformer import HappyTextToText, TTSettings

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

args = TTSettings(num_beams=5, min_length=1)

# Add the prefix "grammar: " before each input 
result = happy_tt.generate_text("grammar: This sentences has has bads grammar.", args=args)

print(result.text) # This sentence has bad grammar.


In [ ]:
# To see the first few results:
for pair in results[500:505]:
    print(pair)

In [ ]:
corrector("fix: posttestinspectionandengineteardownrevealedmechanicalfailure")

In [ ]:
import json
import re
from spellchecker import SpellChecker

# Load the JSON paragraphs file
with open("/Users/rckyi/Documents/Data/paragraphs_with_ids.json", "r", encoding="utf-8") as f:
    paragraphs = json.load(f)

# Initialize the spell checker
spell = SpellChecker()

# Function to attempt splitting a word into valid parts
def split_word(word):
    word = word.lower()
    if word in spell:
        return word
    for i in range(3, len(word) - 3):
        prefix, suffix = word[:i], word[i:]
        if prefix in spell and suffix in spell:
            return prefix + ' ' + suffix
    return word

# Function to correct concatenated words in a paragraph
def correct_paragraph(paragraph):
    corrected_tokens = []
    for token in re.findall(r'\w+|\W+', paragraph):
        if token.strip().isalpha():
            corrected_tokens.append(split_word(token))
        else:
            corrected_tokens.append(token)
    return ''.join(corrected_tokens)

# Apply correction to all paragraphs
corrected_lines = []
for pid, para in paragraphs.items():
    # cleaned = correct_paragraph(para)
    cleaned = corrector(f"fix: +{para}")
    corrected_lines.append(f"{pid}:\n{cleaned}\n")

# print(f'corrected lines {corrected_lines}')

# Write to output text file
with open("/Users/rckyi/Documents/Data/corrected_paragraphs_output.txt", "w", encoding="utf-8") as f:
    f.writelines(corrected_lines)

print("Corrected text saved to 'corrected_paragraphs_output.txt'")


In [ ]:
import json
import re
from spellchecker import SpellChecker

# Load the JSON paragraphs file
with open("/Users/rckyi/Documents/Data/paragraphs_with_ids.json", "r", encoding="utf-8") as f:
    paragraphs = json.load(f)

# Initialize the spell checker
spell = SpellChecker()

# Function to attempt splitting a word into valid parts
def split_word(word):
    word = word.lower()
    if word in spell:
        return word
    for i in range(3, len(word) - 3):
        prefix, suffix = word[:i], word[i:]
        if prefix in spell and suffix in spell:
            return prefix + ' ' + suffix
    return word

# Function to correct concatenated words in a paragraph
def correct_paragraph(paragraph):
    corrected_tokens = []
    for token in re.findall(r'\w+|\W+', paragraph):
        if token.strip().isalpha():
            corrected_tokens.append(split_word(token))
        else:
            corrected_tokens.append(token)
    return ''.join(corrected_tokens)

# Apply correction to all paragraphs
corrected_dict = {}
for pid, para in paragraphs.items():
    corrected_dict[pid] = correct_paragraph(para)

# Save the corrected paragraphs to JSON
with open("/Users/rckyi/Documents/Data/corrected_paragraphs_output.json", "w", encoding="utf-8") as f:
    json.dump(corrected_dict, f, indent=2, ensure_ascii=False)

print("Corrected paragraphs saved to 'corrected_paragraphs_output.json'")


In [ ]:
split_word("posttestinspectionandengineteardownrevealedmechanicalfailure")

In [ ]:
def fetch_impulse_classes():
   
    # URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/Model_rocket_motor_classification"
    
    # Send a request to the webpage
    response = requests.get(url)
    # print(f'response json {response.json()}')
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(f'soup {soup}')
    
    # Find all tables
    tables = soup.find_all('table', {'class': 'wikitable'})
    # imp = soup.find_all('ta')
    # print(f'table {tables}')
    
    # Look for the table that contains the header "Total impulse (N·s)"
    target_table = None
    
    for table in tables:
        # print(f'table text {table.}')
        if 'impulse' in table.text:
            print('found')
            target_table = table
            break
    # print(target_table)
    # Parse the table into a DataFrame
    df = pd.read_html(str(tables))[0]
    
    # Display the resulting DataFrame
    print(df['Class  (Base 26)'])
    ls = df['Class  (Base 26)'].tolist()
    
    return [x for x in ls if len(x) < 10]

In [ ]:
# Stack Exchange API
# def fetch_stackexchange_qa(site='space', tagged='rocket', pages=5):
#     questions = []
#     for page in range(1, pages+1):
#         url = f"https://api.stackexchange.com/2.3/questions?page={page}&pagesize=20&order=desc&sort=activity&tagged={tagged}&site={site}&filter=withbody"
#         resp = requests.get(url).json()
#         print(f'resp {resp}')
#         for item in resp.get('items', []):
#             q = {
#                 'question': item['title'],
#                 'body': item.get('body', ''),
#                 'answers': []
#             }
#             if item.get('is_answered'):
#                 a_url = f"https://api.stackexchange.com/2.3/questions/{item['question_id']}/answers?order=desc&sort=votes&site={site}&filter=withbody"
#                 a_resp = requests.get(a_url).json()
#                 for answer in a_resp.get('items', []):
#                     q['answers'].append(answer['body'])
#                     questions.append(q)
#             # questions.append(q)
#         time.sleep(1)  # Respect rate limits
#     return questions

 
# arXiv API Q&A extraction from abstract (pseudo-QA from title and abstract)
def fetch_arxiv_abstracts(query='rocket propulsion', max_results=10):
    url = f"http://export.arxiv.org/api/query?search_query=all:{query.replace(' ', '+')}&start=0&max_results={max_results}"
    resp = requests.get(url)
    root = ET.fromstring(resp.content)

    qas = []
    ns = {'atom': 'http://www.w3.org/2005/Atom'}  # arXiv uses Atom XML namespace

    for entry in root.findall('atom:entry', ns):
        title = entry.find('atom:title', ns).text.strip()
        
        summary = entry.find('atom:summary', ns).text.strip()
        # Generate question from text
        input_text = f"generate question: {summary}"
        generated_question = qg_pipeline(input_text, max_length=64, do_sample=False)[0]['generated_text']
        
        qas.append({
            'title': title,
            'question': generated_question,
            'answer': summary
        })
    return qas


In [ ]:
# Wikipedia scraping

# Initialize the question generation pipeline
# qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

def clean_text(text):
    return ' '.join(text.strip().split())

def get_all_rocket_propulsion_links(base_url='https://en.wikibooks.org/wiki/Rocket_Propulsion'):
    """Grab all unique subpage links under Rocket Propulsion."""
    resp = requests.get(base_url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    content_div = soup.select_one('#mw-content-text')
    links = content_div.find_all('a', href=True)
    
    urls = set()
    for link in links:
        href = link['href']
        if 'https' in href:
            full_url = href
        else:
            full_url = 'https://en.wikibooks.org' + href
            # print(f'full_url {full_url}')
        urls.add(full_url)
        # if href.startswith('/wiki/Rocket_Propulsion/') and ':' not in href:
        #     print(f'url: https://en.wikibooks.org + { href}')
        #     urls.add('https://en.wikibooks.org' + href)

    return list(urls)

def scrape_pages_with_qg(urls, visited, M, batch_size=10):
    """Scrape up to `batch_size` new pages not in `visited`, return new QAs."""
    qa_pairs = []
    count = 0
    print(f'visited {visited}')

    for url in urls:
        if url in visited:
            continue
        visited.add(url)
        try:
            resp = requests.get(url)
            soup = BeautifulSoup(resp.text, 'html.parser')
            content_div = soup.select_one('#mw-content-text')
            paragraphs = content_div.find_all('p')
            text = clean_text(' '.join(p.get_text() for p in paragraphs[:3]))
            if not text or len(text) < M:
                continue

            # Generate question from text
            input_text = f"generate question: {text}"
            output = qg_pipeline(input_text, max_length=64, do_sample=False)[0]['generated_text']

            qa_pairs.append({
                'question': output,
                'answer': text,
                'source_url': url
            })
            print(f"✅ Generated Q&A from: {url}")
            count += 1
            time.sleep(1)  # polite scraping

            if count >= batch_size:
                break
        except Exception as e:
            print(f"⚠️ Failed to process {url}: {e}")
    return qa_pairs

def scrape_wikibook_qas(M):
    all_links = get_all_rocket_propulsion_links()
    visited = set()
    all_qas = []

    for i in range(10):
        print(f"\n🔁 Batch {i+1}/10")
        batch_qas = scrape_pages_with_qg(all_links, visited, M, batch_size=10)
        all_qas.extend(batch_qas)
        if len(all_qas) >= M:
            break

    # with open('wikibook_qas_100.json', 'w', encoding='utf-8') as f:
    #     json.dump(all_qas, f, indent=2)

    print(f"\n✅ Done. Saved {len(all_qas)} Q&A pairs to wikibook_qas_100.json")
    return all_qas

In [ ]:
# Stack Exchange API

def clean_html_text(html_content):
    """Remove hyperlinks and strip HTML tags from content."""
    soup = BeautifulSoup(html_content, 'html.parser')

    # Replace <a> tags with their inner text
    for a in soup.find_all('a'):
        a.replace_with(a.get_text())

    # Get cleaned text
    text = soup.get_text(separator=' ')
    return html.unescape(text.strip())

def fetch_stackexchange_qa_data(site='space.stackexchange', tag='rockets', pagesize=20, max_pages=2):
    base_url = 'https://api.stackexchange.com/2.3/questions'
    answers_url = 'https://api.stackexchange.com/2.3/questions/{ids}/answers'
    all_qas = []

    for page in range(1, max_pages + 1):
        params = {
            'site': site,
            'tagged': tag,
            'pagesize': pagesize,
            'page': page,
            'filter': 'withbody'
        }
        resp = requests.get(base_url, params=params)
        data = resp.json()

        for question in data.get('items', []):
            q_id = question['question_id']
            q_body = clean_html_text(question.get('body', ''))
            title = html.unescape(question.get('title', ''))

            # Get answers
            a_params = {
                'site': site,
                'filter': 'withbody'
            }
            a_resp = requests.get(answers_url.format(ids=q_id), params=a_params)
            answers = a_resp.json().get('items', [])

            for ans in answers:
                a_body = clean_html_text(ans.get('body', ''))
                all_qas.append({
                    'question': f"{title}\n{q_body}",
                    'answer': a_body,
                    'source_url': f"https://{site}.com/questions/{q_id}"
                })

            time.sleep(1)  # polite API usage

    return all_qas


In [ ]:
if __name__ == "__main__":
    se_data = fetch_stackexchange_qa_data()
    # arxiv_data = fetch_arxiv_abstracts()
    # reddit_data = fetch_reddit_qa()
    # wikibook_data = scrape_wikibook_qas(M=100)

    # all_qas = {
    #     "stackexchange": se_data,
    #     "arxiv": arxiv_data,
    #     "reddit": reddit_data,
    #     "wikibook": wikibook_data
    # }

    # save_qas_to_file(all_qas)
    # print("Data collection complete. Saved to qa_data.json")


In [ ]:
# se_data

In [ ]:
se_data[12]['question']#.split('\n')[0]

In [ ]:
se_data[12]['answer']

In [ ]:
se_data[1]["body"].replace("<p>","").replace('"','').replace("</blockquote>"," ").replace("<a>","").replace("<blockquote>"," ")